In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importation

In [ ]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



fpath = "/kaggle/input/heart-failure-prediction/heart.csv"
df = pd.read_csv(fpath)
df.shape

In [ ]:
df.head()

Binary classification task

In [ ]:
df.describe()

## Preparation

In [ ]:
le = LabelEncoder()
dict_le = {}
cats = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
for cat in cats:
    dict_le[cat] = LabelEncoder()
    df[f"{cat}_encoded"] = dict_le[cat].fit_transform(df[cat])
    
df = df.rename(columns={"HeartDisease": "target"})

In [ ]:
df.head()

In [ ]:
df_processed = df.drop(cats, axis=1)
train, test = train_test_split(df_processed, test_size=0.2)

In [ ]:
X_train = train.drop("target", axis=1).values
y_train = train.target.values

X_test = test.drop("target", axis=1).values
y_test = test.target.values

## Random Forest Classifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)
y_pred

In [ ]:
np.mean(y_train == y_pred)

Trying crossvalidation

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
np.mean(y_test == y_pred)

In [ ]:
X_train

We are overfitting a bit (100% on train set vs 86.41% accuracy on test set)

In [ ]:
y_train

## SVM Classifier

In [ ]:
model = SVC()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)
np.mean(y_train == y_pred)

In [ ]:
y_pred = model.predict(X_test)
np.mean(y_test == y_pred)

## XGBoost Classifier

In [ ]:

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
np.mean(y_train == y_pred)

In [ ]:
y_pred = model.predict(X_test)
np.mean(y_test == y_pred)

XGBoost is yielding to the best accuracy without optimizing any hyperparameters and with not much preprocessing

## Keras model

In [ ]:
# baseline model
def create_baseline():
    model = Sequential()
    model.add(Dense(60, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = create_baseline()
model.fit(X_train, y_train, epochs=100, validation_split=0.1)

In [ ]:
y_pred = model.predict(X_test)
y_pred = [1 if y > 0.5 else 0 for y in y_pred]
np.mean(y_test == y_pred)

Using these hyperparameters, keras model is not better than XGBoost on this dataset.